# **Data Collectors and Landing Zone**

### *Required packages:*

In [1]:
 !pip install opendatasets
 !pip install kaggle
 !pip install openpyxl
 !pip install pyarrow

In [2]:
import numpy as np
import pandas as pd
import requests
import os
import opendatasets as od
import csv

### Kaggle Configuration - Drive

(Requereix del fitxer `kaggle_fitxer.json`)

In [3]:
!mkdir ~/.kaggle

In [4]:
!wget https://raw.githubusercontent.com/OscarMoliina/betterlifebetterhealth/main/src/utils/kaggle_user_config.json

--2024-04-22 18:46:49--  https://raw.githubusercontent.com/OscarMoliina/betterlifebetterhealth/main/src/utils/kaggle_user_config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73 [text/plain]
Saving to: ‘kaggle_user_config.json’

kaggle_user_config. 100%[===================>]      73  --.-KB/s    in 0s      

2024-04-22 18:46:49 (1.31 MB/s) - ‘kaggle_user_config.json’ saved [73/73]



In [5]:
!cp ./kaggle_user_config.json ~/.kaggle/kaggle.json

In [6]:
!cd ~/.kaggle && ls

kaggle.json


### Downloading datasets

In [7]:
#CREAR DIRECTORIOS SI NO EXISTEN:

# Nombre del directorio
nombre_directorio = "data"

# Verificar si el directorio existe
if not os.path.exists(nombre_directorio):
    # Crear el directorio
    os.makedirs(nombre_directorio)
    print(f"Directorio '{nombre_directorio}' creado exitosamente.")
else:
    print(f"El directorio '{nombre_directorio}' ya existe.")

%cd data

nombre_directorio = "csv"

# Verificar si el directorio existe
if not os.path.exists(nombre_directorio):
    # Crear el directorio
    os.makedirs(nombre_directorio)
    print(f"Directorio '{nombre_directorio}' creado exitosamente.")
else:
    print(f"El directorio '{nombre_directorio}' ya existe.")

nombre_directorio = "parquet"

# Verificar si el directorio existe
if not os.path.exists(nombre_directorio):
    # Crear el directorio
    os.makedirs(nombre_directorio)
    print(f"Directorio '{nombre_directorio}' creado exitosamente.")
else:
    print(f"El directorio '{nombre_directorio}' ya existe.")

%cd ..

Directorio 'data' creado exitosamente.
/content/data
Directorio 'csv' creado exitosamente.
Directorio 'parquet' creado exitosamente.
/content


##### UNECE

In [8]:
# URL del dataset UNECE
unece_db = pd.read_json('https://query.data.world/s/qd6kjqguuvb573mffbpfysgpudi2pv?dws=00000')
unece_db.to_csv('unece_db.csv', index=False)
!mv unece_db.csv data/csv/unece_db.csv

##### Kaggle - Mental Health

In [9]:
# API Kaggle Mental Health
!kaggle datasets download  -d 'thedevastator/uncover-global-trends-in-mental-health-disorder'

 81% 1.00M/1.24M [00:00<00:00, 1.51MB/s]
100% 1.24M/1.24M [00:00<00:00, 1.78MB/s]


In [10]:
#!sudo apt-get install unzip
!unzip uncover-global-trends-in-mental-health-disorder.zip -d data/csv
!rm uncover-global-trends-in-mental-health-disorder.zip

Archive:  uncover-global-trends-in-mental-health-disorder.zip
  inflating: data/csv/Mental health Depression disorder Data.csv  


##### USA Census

In [ ]:
# API United States Census
variables = "NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas"
url_base = "https://api.census.gov/data/timeseries/idb/5year?get=NAME,YR,AREA_KM2,CBR,CDR,DEATHS,E0,GRR,IMR,MEDAGE,MR0_4,POP_DENS&for=genc%20standard%20countries%20and%20areas:*".format(variables)

# Llista per guardar les dades
data = []

# Fem una sol·licitud a l'API per agafar dades des de l'any 1990 fins al 2019
for year in range(1990, 2019):
    url = "{}&time={}".format(url_base, year)
    response = requests.get(url)
    if response.status_code == 200:
        data.extend(response.json()[1:])

# Guardar los datos en un archivo CSV
csv_filename = "dades_demogr_census.csv"
with open(csv_filename, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas", "code"])
    csv_writer.writerows(data)

print("Les dades s'han guardat a", csv_filename)

!mv dades_demogr_census.csv data/csv/dades_demogr_census.csv


#### Rename

In [ ]:
!mv data/csv/dades_demogr_census.csv data/csv/demography.csv
!mv data/csv/unece_db.csv data/csv/society.csv
os.rename("data/csv/Mental health Depression disorder Data.csv", "data/csv/mental_health.csv")


#### Storing files in a suitable format (Parquet)


In [ ]:
# Unece_db
unece_db = pd.read_csv('data/csv/society.csv')
unece_db.to_parquet('data/parquet/society.parquet')

# Kaggle db
kaggle_db = pd.read_csv('data/csv/mental_health.csv', decimal=',')
kaggle_db.to_parquet('data/parquet/mental_health.parquet')

# United State Census db
us_census_db = pd.read_csv('data/csv/demography.csv')
us_census_db.to_parquet('data/parquet/demography.parquet')